The problem with the query parameters
The SQL representation of many data types is often different from their Python string representation. The typical example is with single quotes in strings: in SQL single quotes are used as string literal delimiters, so the ones appearing inside the string itself must be escaped, whereas in Python single quotes can be left unescaped if the string is delimited by double quotes.

Because of the difference, sometime subtle, between the data types representations, a naïve approach to query strings composition, such as using Python strings concatenation, is a recipe for terrible problems:

https://www.psycopg.org/docs/usage.html#the-problem-with-the-query-parameters

https://www.psycopg.org/docs/sql.html
psycopg2.sql – SQL string composition¶
New in version 2.7.

The module contains objects and functions useful to generate SQL dynamically, in a convenient and safe way. SQL identifiers (e.g. names of tables and fields) cannot be passed to the execute() method like query arguments:

This is the only way to write effective and safe sql queries from python!

SPEND A LOT OF TIME COVERING THIS MODEL IN THE FINAL PRESENTATION

Numbers adaptation
Python numeric objects int, long, float, Decimal are converted into a PostgreSQL numerical representation:

>>> cur.mogrify("SELECT %s, %s, %s, %s;", (10, 10L, 10.0, Decimal("10.00")))
'SELECT 10, 10, 10.0, 10.00;'


Reading from the database, integer types are converted into int, floating point types are converted into float, numeric/decimal are converted into Decimal.

Talk about this function

cur.mogrify("SELECT %s, %s, %s, %s;", (10, 10L, 10.0, Decimal("10.00")))
'SELECT 10, 10, 10.0, 10.00;'

In [1]:
import psycopg2, os, getpass


In [2]:
def create_server_connection(database, user):
    conn = None
    try:
        conn = psycopg2.connect(
            database=database,
            user=user,
        )
        print(f"Connected to Database {database} successfully")
        
    except Exception as e:
        print(e)

    return conn

In [3]:
def create_or_drop_database(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    with conn.cursor() as cur:
        try:
            cur.execute(query)
            print(f"Query: {query} ran successfully.")
        except Exception as e:
            print(e)
        finally:
            #be a good citizen and put stuff back the way you found it
            conn.autocommit = autocommit 

In [15]:
# let's write an execute query function
# we want our function to return a cursor with the results

#version 1
def execute_query(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    cur = conn.cursor()
    try:
        cur.execute(query)
        print(f"Query: {query}ran successfully.")
    except Exception as e:
        print(e)
    finally:
        #be a good citizen and put stuff back the way you found it
        conn.autocommit = True 
    return cur

In [16]:
db, user = "jupyterdb", getpass.getuser()
conn = create_server_connection(db,user)

Connected to Database jupyterdb successfully


In [6]:
conn.autocommit = True

In [ ]:
# let's create an authors table in our database

In [10]:
query = """
DROP TABLE IF EXISTS author;

CREATE TABLE author(
name text,
email text
);
"""
execute_query(conn, query)

Query: 
DROP TABLE IF EXISTS author;
CREATE TABLE author(
name text,
email text
);
ran successfully.


<cursor object at 0x10d58ae50; closed: 0>

In [12]:
# DON'T DO THIS

SQL = "INSERT INTO author (name) VALUES ('%s');" # NEVER DO THIS
data = ("O'Reilly", )
cur = execute_query(conn, SQL % data) # THIS WILL FAIL MISERABLY

syntax error at or near "Reilly"
LINE 1: INSERT INTO author (name) VALUES ('O'Reilly');
                                             ^



In [ ]:
# DO THIS INSTEAD

In [14]:
SQL = "INSERT INTO author (name) VALUES (%s);" # Note: no quotes
>>> data = ("O'Reilly", )
cur = conn.cursor()
>>> cur.execute(SQL, data) # Note: no % operator

In [ ]:
#now let's change our execute_query function accordingly

In [28]:
#version 2
def execute_query(conn, *query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    cur = conn.cursor()
    try:
        if query[1]:
            cur.execute(*query)
            print(f"Query: {query[0]} with data:{query[1]} ran successfully.")
        else:
            cur.execute(query[0])
            print(f"Query: {query[0]} ran successfully.")
    except Exception as e:
        print(e)
    finally:
        #be a good citizen and put stuff back the way you found it
        conn.autocommit = True 
    return cur

In [29]:
SQL = "INSERT INTO author (name) VALUES (%s);" # Note: no quotes
data = ("O'Reilly", )
cur = execute_query(conn, SQL, data)

Query: INSERT INTO author (name) VALUES (%s); with data:("O'Reilly",) ran successfully.
